Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy_, valamint a _scipy.stats t_ és _f_ osztályai

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path


In [ ]:
datalib = Path.cwd().parent.joinpath('data')

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot.
 <u>Ebben a blokkban önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályodat.</u>

7., Készíts egy új __LinearRegressionNP__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami OLS elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között. <u>__(numpy.linalg.lstsq() nem használható)__</u>

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítsd ki az osztályt egy __get_wald_test_result__ metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Wald: wald_value, p-value: p_value__, ahol az wald_value és p_value helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. A függvény bemenete során feltételezzük, hogy r minden eleme 0, R-t listák listája formában adjuk át. A Wald statisztika p értékét az előző feladathoz hasonlóan számold ki, de figyelj, hogy a <u>teszt 1 oldalú</u>.

12., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np

sp500=pd.read_parquet('../data/sp500.parquet')

ff_factors=pd.read_parquet('../data/ff_factors.parquet')

data=pd.merge(sp500, ff_factors, on='Date', how='left')

data['Excess Return']=data['Monthly Returns'] - data['RF']

data.sort_index(inplace=True)
data['ex_ret_1']=data.groupby('Symbol')['Excess Return'].shift(-1)
data.sort_values(['Symbol', 'Date'], ascending=[True, True], inplace=True)

data.dropna(subset=['ex_ret_1'], inplace=True)
data.dropna(subset=['HML'], inplace=True)

data=data[data['Symbol']=='AMZN']
data.drop(columns=['Symbol'], inplace=True)

In [3]:
class LinearRegressionNP:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self.coefficients = None
        self.residuals = None
        self.p_values = None
        self.crs = None
        self.ars = None

    def fit(self):
        X = np.column_stack((np.ones(len(self.right_hand_side)), self.right_hand_side))
        y = self.left_hand_side
        self.coefficients = np.linalg.inv(X.T @ X) @ X.T @ y

        n, k = X.shape
        self.residuals = y - X @ self.coefficients
        sigma_squared = np.sum(self.residuals ** 2) / (n - k)
        var_beta = np.linalg.inv(X.T @ X) * sigma_squared
        t_statistic = self.coefficients / np.sqrt(np.diag(var_beta))
        self.p_values = 2 * (1 - stats.t.cdf(np.abs(t_statistic), df=n-k))

        rss = np.sum(self.residuals ** 2)
        tss = np.sum((y - np.mean(y)) ** 2)

        self.crs = 1 - rss / tss
        self.ars = 1 - (rss / (n - k)) / (tss / (n - 1))
    def get_params(self):
        return pd.Series(self.coefficients, name='Beta coefficients')

    def get_pvalues(self):
        return pd.Series(self.p_values, name='P-values for the corresponding coefficients')

    def get_wald_test_result(self, constraints):
        constraints = np.array(constraints)
        n = len(self.left_hand_side)
        m, k = constraints.shape
        sigma_squared = np.sum(self.residuals ** 2) / (n - k)
        X = np.column_stack((np.ones(len(self.right_hand_side)), self.right_hand_side))

        H = constraints @ np.linalg.inv(X.T @ X) @ constraints.T
        wald = (constraints @ self.coefficients).T @ np.linalg.inv(H) @ (constraints @ self.coefficients)
        wald = wald / m / sigma_squared
        p_value = 1 - stats.f.cdf(wald, dfn=m, dfd=n - k)

        return f'Wald: {wald:.3f}, p-value: {p_value:.3f}'

    def get_model_goodness_values(self):
        crs = self.crs
        ars = self.ars

        return f'Centered R-squared: {crs:.3f}, Adjusted R-squared: {ars:.3f}'


In [4]:
model = LinearRegressionNP(data["ex_ret_1"], data[["Mkt-RF", "SMB", "HML"]])
model.fit()
print(model.get_params())
print(model.get_pvalues())
print(model.get_wald_test_result([[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]))
print(model.get_model_goodness_values())

0    0.015820
1   -0.471831
2    0.411126
3   -0.054560
Name: Beta coefficients, dtype: float64
0    0.104176
1    0.026106
2    0.173041
3    0.846058
Name: P-values for the corresponding coefficients, dtype: float64
Wald: 1.979, p-value: 0.119
Centered R-squared: 0.030, Adjusted R-squared: 0.015
